In [1]:
# parcel_hyperalignment.py
# runs hyperalignment on each parcel individually, saves common spaces and mappers
import utils
import numpy as np
import os, sys, glob, time
import multiprocessing as mp
from mvpa2.mappers.zscore import zscore
from mvpa2.algorithms.hyperalignment import Hyperalignment
from mvpa2.datasets import Dataset
from mvpa2.base import debug
from datetime import timedelta

NPROC = 24
os.environ['TMPDIR']='/dartfs-hpc/scratch/f0040y1'
os.environ['TEMP']='/dartfs-hpc/scratch/f0040y1'
os.environ['TMP']='/dartfs-hpc/scratch/f0040y1'
TEMPORARY_OUTDIR='/dartfs-hpc/scratch/f0040y1'

In [2]:
def get_subject_list(nsubs=None):
    # load files 
    in_train_files = glob.glob(os.path.join(utils.sponpain_by_parcel, '*'))
    in_train_subs = np.unique([os.path.basename(x)[0:13] for x in in_train_files])

    in_test_files = glob.glob(os.path.join(utils.bladderpain_by_parcel, '*'))
    in_test_subs = np.unique([os.path.basename(x)[0:13] for x in in_test_files])

    bothsubs = [sub for sub in in_train_subs if sub in in_test_subs]

    if nsubs is not None:
        bothsubs = bothsubs[:nsubs]
    return bothsubs

In [16]:
def apply_mappers((data_fn, mapper_fn, subject, mapper, parcel_num)):
    data = utils.prep_parcelwise_data(subject, parcel_num, 'bladderpain')
    aligned = zscore((np.asmatrix(data)*mapper._proj).A, chunks_attr=None)
    np.save(data_fn, aligned)
    np.savez(mapper_fn, mapper)


In [4]:
parcel_num = 1
# get list of subjects to hyperalign
sub_list = get_subject_list()

aligned_dirname = os.path.join(utils.common_space_dir, 'parcel_{n:03d}'.format(n=parcel_num))
mapper_dirname = os.path.join(utils.trans_matrices, 'parcel_{n:03d}'.format(n=parcel_num))

for d in [aligned_dirname, mapper_dirname]:
    if not os.path.exists(d):
        os.makedirs(d)

In [5]:
train_dss = [utils.prep_parcelwise_data(sub, parcel_num, 'sponpain') for sub in sub_list]

In [8]:
ha = Hyperalignment(nproc=NPROC, joblib_backend='multiprocessing')

In [9]:
debug.active += ['HPAL']
t0 = time.time()
ha.train(train_dss)
mappers = ha(train_dss)
t1 = time.time()
print('-------- done training hyperalignment at {B} --------'.format(B=str(timedelta(seconds=t1-t0))))

[HPAL] DBG:                    Hyperalignment Hyperalignment: 3 ca:ca{residual_errors chosen_ref_ds+ training_residual_errors} 12 params:params{joblib_backend=multiprocessing output_dim=None zscore_common=True level2_niter=1...} for 202 datasets
 * Please note: warnings are printed only once, but underlying problem might occur many times *
-------- done training hyperalignment at 0:02:11.704165 --------


In [12]:
print('length of test subjects= '+str(len(train_dss)))
data_fns = [os.path.join(aligned_dirname,'{s}_aligned_dtseries.npy'.format(s=s)) for s in sub_list]
mapper_fns = [os.path.join(mapper_dirname,'{s}_trained_mapper.npz'.format(s=s)) for s in sub_list]
iterable = zip(data_fns, mapper_fns, sub_list, mappers, np.repeat(parcel_num, len(mappers)))

length of test subjects= 202


In [18]:
apply_mappers(iterable[0])

In [17]:
iterable[0]

('/dartfs-hpc/rc/home/1/f0040y1/CANlab/labdata/projects/OLP4CBP/hyperalignment/parcelwise/data/bladder_raw_in_spon/parcel_001/sub-M80302134_aligned_dtseries.npy',
 '/dartfs-hpc/rc/home/1/f0040y1/CANlab/labdata/projects/OLP4CBP/hyperalignment/parcelwise/transformation_matrices/parcel_001/sub-M80302134_trained_mapper.npz',
 'sub-M80302134',
 ProcrusteanMapper(space='commonspace', oblique_rcond=-1.0),
 1)

In [19]:
for i in range(487):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27